<a href="https://colab.research.google.com/github/zakariazemmahi/waste-detection-yolov8/blob/main/Models/Application_de_comptur_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **⚙️ Étape 1 – Installer les dépendances**

In [2]:
!pip install streamlit ultralytics
!npm install -g localtunnel


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

# **📝 Étape 2 – Créer ton fichier app.py Streamlit**

In [3]:
%%writefile app.py
import streamlit as st
from ultralytics import YOLO
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import requests
import streamlit.components.v1 as components
import cv2
from io import BytesIO
import base64

# ------------------------
# Configuration de la page
# ------------------------
st.set_page_config(
    page_title="SmartWasteDetection",
    page_icon="♻️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ------------------------
# CSS personnalisé pour un design moderne
# ------------------------
st.markdown("""
<style>
    .main-header {
        background: linear-gradient(90deg, #4CAF50, #81C784);
        padding: 2rem;
        border-radius: 10px;
        text-align: center;
        margin-bottom: 2rem;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }

    .main-header h1 {
        color: white;
        font-size: 3rem;
        margin: 0;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.3);
    }

    .main-header p {
        color: white;
        font-size: 1.2rem;
        margin: 0.5rem 0 0 0;
        opacity: 0.9;
    }

    .detection-card {
        background: white;
        padding: 1.5rem;
        border-radius: 10px;
        box-shadow: 0 2px 10px rgba(0, 0, 0, 0.1);
        margin: 1rem 0;
        border-left: 4px solid #4CAF50;
    }

    .waste-detected {
        background: linear-gradient(90deg, #E8F5E8, #F1F8E9);
        border-left-color: #4CAF50;
    }

    .no-waste {
        background: linear-gradient(90deg, #FFF3E0, #FFF8E1);
        border-left-color: #FF9800;
    }

    .classification-result {
        background: #E3F2FD;
        padding: 1rem;
        border-radius: 8px;
        margin: 0.5rem 0;
        border-left: 3px solid #2196F3;
    }

    .stats-container {
        display: flex;
        justify-content: space-around;
        margin: 1rem 0;
    }

    .stat-item {
        text-align: center;
        padding: 1rem;
        background: #F5F5F5;
        border-radius: 8px;
        min-width: 120px;
    }

    .sidebar-info {
        background: #F8F9FA;
        padding: 1rem;
        border-radius: 8px;
        margin: 1rem 0;
    }
</style>
""", unsafe_allow_html=True)

# ------------------------
# Animation Lottie
# ------------------------
@st.cache_data
def load_lottie_url(url):
    try:
        r = requests.get(url)
        if r.status_code == 200:
            return r.json()
    except:
        pass
    return None

# ------------------------
# Fonctions utilitaires
# ------------------------
@st.cache_resource
def load_models():
    """Chargement des modèles YOLO avec mise en cache"""
    try:
        model_detect = YOLO("/content/drive/MyDrive/yolov8_best_smartdetection.pt")
        model_classify = YOLO("/content/drive/MyDrive/yolov8_best.pt")
        return model_detect, model_classify, True
    except Exception as e:
        st.error(f"Erreur lors du chargement des modèles : {str(e)}")
        return None, None, False

def process_image_detection(image, model_detect):
    """Traite la détection de déchets sur l'image"""
    img_array = np.array(image)

    # Prédiction avec le modèle de détection
    results = model_detect.predict(img_array, conf=0.25, verbose=False)

    waste_objects = []
    non_waste_objects = []

    if results and len(results) > 0:
        result = results[0]

        for box in result.boxes:
            cls_id = int(box.cls)
            class_name = result.names[cls_id]
            confidence = float(box.conf)
            bbox = box.xyxy.cpu().numpy().astype(int)[0]

            detection_info = {
                'class': class_name,
                'confidence': confidence,
                'bbox': bbox
            }

            if class_name.lower() == "dechet":
                waste_objects.append(detection_info)
            else:
                non_waste_objects.append(detection_info)

    return waste_objects, non_waste_objects, results[0] if results else None

def classify_waste(image, bbox, model_classify):
    """Classifie le type de déchet détecté"""
    img_array = np.array(image)
    x1, y1, x2, y2 = bbox

    # Extraction de la région d'intérêt
    cropped = img_array[y1:y2, x1:x2]

    if cropped.size == 0:
        return None, 0

    # Classification du déchet
    results = model_classify.predict(cropped, conf=0.25, verbose=False)

    if results and len(results) > 0 and len(results[0].boxes) > 0:
        best_box = results[0].boxes[0]  # Prendre la meilleure détection
        class_id = int(best_box.cls)
        class_name = results[0].names[class_id]
        confidence = float(best_box.conf)
        return class_name, confidence

    return None, 0

def create_annotated_image(image, waste_objects, non_waste_objects, classifications=None):
    """Crée une image annotée avec les détections"""
    img_draw = image.copy()
    draw = ImageDraw.Draw(img_draw)

    # Couleurs pour les différents types
    waste_color = (255, 0, 0)  # Rouge pour déchets
    non_waste_color = (0, 255, 0)  # Vert pour non-déchets

    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 20)
    except:
        font = ImageFont.load_default()

    # Annoter les déchets
    for i, obj in enumerate(waste_objects):
        bbox = obj['bbox']
        draw.rectangle(bbox, outline=waste_color, width=3)

        label = f"Déchet ({obj['confidence']:.2f})"
        if classifications and i < len(classifications) and classifications[i]:
            waste_type, conf = classifications[i]
            if waste_type:
                label += f"\n{waste_type} ({conf:.2f})"

        draw.text((bbox[0], bbox[1] - 30), label, fill=waste_color, font=font)

    # Annoter les non-déchets
    for obj in non_waste_objects:
        bbox = obj['bbox']
        draw.rectangle(bbox, outline=non_waste_color, width=3)
        label = f"{obj['class']} ({obj['confidence']:.2f})"
        draw.text((bbox[0], bbox[1] - 30), label, fill=non_waste_color, font=font)

    return img_draw

# ------------------------
# Interface Streamlit
# ------------------------

# Barre latérale
with st.sidebar:
    st.markdown("""
    <div class="sidebar-info">
        <div style="text-align: center;">
            <img src="https://cdn-icons-png.flaticon.com/512/10067/10067108.png" width="80">
            <h2>♻️ Smart Waste Detection</h2>
        </div>
    </div>
    """, unsafe_allow_html=True)

    st.markdown("### 🎯 À propos du projet")
    st.markdown("""
    Ce système utilise l'intelligence artificielle pour :

    **🔍 Étape 1 :** Détecter les objets dans l'image
    - Identifier si un objet est un déchet ou non

    **🗂️ Étape 2 :** Classifier les déchets détectés
    - Déterminer le type de déchet (plastique, verre, etc.)
    """)

    st.markdown("---")
    st.markdown("### 👥 Équipe")
    st.markdown("""
    - **Zakariae Zemmahi**
    - **Mohamed Hajji**
    - **Ess_aidi Youssef**
    """)

    st.markdown("---")
    st.markdown("📍 **ENSAM Meknès**  \n📅 **Mai 2025**")

# En-tête principal
st.markdown("""
<div class="main-header">
    <h1>🌍 Smart Waste Detection</h1>
    <p>Détection intelligente et classification automatique des déchets</p>
</div>
""", unsafe_allow_html=True)

# Animation Lottie
lottie_url = "https://assets7.lottiefiles.com/packages/lf20_qp1q7mct.json"
lottie_json = load_lottie_url(lottie_url)

if lottie_json:
    components.html(f"""
        <div style="display: flex; justify-content: center; margin: 2rem 0;">
            <lottie-player src="{lottie_url}" background="transparent" speed="1"
                         style="width: 200px; height: 200px;" loop autoplay>
            </lottie-player>
        </div>
        <script src="https://unpkg.com/@lottiefiles/lottie-player@latest/dist/lottie-player.js"></script>
    """, height=220)

# Chargement des modèles
with st.spinner("🔄 Chargement des modèles IA..."):
    model_detect, model_classify, models_loaded = load_models()

if not models_loaded:
    st.error("❌ Impossible de charger les modèles. Vérifiez les chemins des fichiers.")
    st.stop()

st.success("✅ Modèles chargés avec succès !")

# Section d'upload
st.markdown("### 📤 Importation des images")
st.markdown("Sélectionnez une ou plusieurs images pour analyser la présence de déchets")

uploaded_files = st.file_uploader(
    "Choisir des images",
    type=["jpg", "jpeg", "png"],
    accept_multiple_files=True,
    help="Formats supportés : JPG, JPEG, PNG"
)

if uploaded_files:
    # Statistiques globales
    total_images = len(uploaded_files)
    total_waste_detected = 0
    total_objects_detected = 0

    st.markdown(f"### 📊 Analyse de {total_images} image(s)")

    # Traitement de chaque image
    for idx, uploaded_file in enumerate(uploaded_files):
        st.markdown("---")
        st.markdown(f"### 🖼️ Image {idx + 1}: `{uploaded_file.name}`")

        # Chargement de l'image
        image = Image.open(uploaded_file).convert("RGB")

        col1, col2 = st.columns(2)

        with col1:
            st.image(image, caption="Image originale", use_column_width=True)

        with st.spinner("🔍 Analyse en cours..."):
            # Étape 1: Détection déchet/non-déchet
            waste_objects, non_waste_objects, detection_result = process_image_detection(image, model_detect)

            # Statistiques pour cette image
            total_objects_in_image = len(waste_objects) + len(non_waste_objects)
            total_objects_detected += total_objects_in_image

            # Étape 2: Classification des déchets détectés
            classifications = []
            if waste_objects:
                total_waste_detected += len(waste_objects)

                for waste_obj in waste_objects:
                    waste_type, conf = classify_waste(image, waste_obj['bbox'], model_classify)
                    classifications.append((waste_type, conf))

            # Création de l'image annotée
            annotated_image = create_annotated_image(image, waste_objects, non_waste_objects, classifications)

        with col2:
            st.image(annotated_image, caption="Résultats de détection", use_column_width=True)

        # Résultats détaillés
        if waste_objects:
            st.markdown(f"""
            <div class="detection-card waste-detected">
                <h4>♻️ Déchets détectés : {len(waste_objects)}</h4>
            </div>
            """, unsafe_allow_html=True)

            for i, (waste_obj, (waste_type, conf)) in enumerate(zip(waste_objects, classifications)):
                if waste_type:
                    st.markdown(f"""
                    <div class="classification-result">
                        <strong>Déchet #{i+1}:</strong> {waste_type}
                        <span style="color: #666;">(Confiance: {conf:.1%})</span>
                    </div>
                    """, unsafe_allow_html=True)
                else:
                    st.markdown(f"""
                    <div class="classification-result">
                        <strong>Déchet #{i+1}:</strong> Type non identifié
                    </div>
                    """, unsafe_allow_html=True)
        else:
            st.markdown(f"""
            <div class="detection-card no-waste">
                <h4>🚫 Aucun déchet détecté</h4>
                <p>Cette image ne contient pas de déchets identifiables.</p>
            </div>
            """, unsafe_allow_html=True)

        # Objets non-déchets détectés
        if non_waste_objects:
            st.markdown(f"**🔍 Autres objets détectés :** {len(non_waste_objects)}")
            objects_list = [f"{obj['class']} ({obj['confidence']:.1%})" for obj in non_waste_objects]
            st.markdown(f"*{', '.join(objects_list)}*")

    # Statistiques finales
    st.markdown("---")
    st.markdown("### 📈 Statistiques globales")

    col1, col2, col3, col4 = st.columns(4)

    with col1:
        st.markdown(f"""
        <div class="stat-item">
            <h3>{total_images}</h3>
            <p>Images analysées</p>
        </div>
        """, unsafe_allow_html=True)

    with col2:
        st.markdown(f"""
        <div class="stat-item">
            <h3>{total_objects_detected}</h3>
            <p>Objets détectés</p>
        </div>
        """, unsafe_allow_html=True)

    with col3:
        st.markdown(f"""
        <div class="stat-item">
            <h3>{total_waste_detected}</h3>
            <p>Déchets trouvés</p>
        </div>
        """, unsafe_allow_html=True)

    with col4:
        waste_percentage = (total_waste_detected / max(total_objects_detected, 1)) * 100
        st.markdown(f"""
        <div class="stat-item">
            <h3>{waste_percentage:.1f}%</h3>
            <p>Taux de déchets</p>
        </div>
        """, unsafe_allow_html=True)

else:
    st.info("👆 Importez une ou plusieurs images pour commencer l'analyse")

    # Instructions d'utilisation
    st.markdown("### 📋 Comment utiliser le système")

    col1, col2, col3 = st.columns(3)

    with col1:
        st.markdown("""
        **🔍 Étape 1**

        Importez vos images à analyser
        """)

    with col2:
        st.markdown("""
        **🤖 Étape 2**

        Le système détecte les déchets automatiquement
        """)

    with col3:
        st.markdown("""
        **📊 Étape 3**

        Classification du type de déchet trouvé
        """)

# Pied de page
st.markdown("---")
st.markdown("""
<div style="text-align: center; padding: 2rem; background: #F8F9FA; border-radius: 10px; margin-top: 2rem;">
    <p style="margin: 0; color: #666;">© 2025 - SmartWasteDetection - ENSAM Meknès</p>
    <p style="margin: 0.5rem 0 0 0; color: #888; font-size: 0.9rem;">
        Propulsé par YOLOv8 et Streamlit 🚀
    </p>
</div>
""", unsafe_allow_html=True)

Writing app.py


# **🌐 Étape 3 – Obtenir ton IP publique (optionnel)**

In [4]:
!wget -q -O - ipv4.icanhazip.com


34.125.255.86


# **🚀 Étape 4 – Lancer l'application et ouvrir un tunnel**

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦

your url is: https://tangy-donuts-push.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.255.86:8501

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
2025-06-12 15:09:06.845 Examining the path of torch.classes raised:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/streamlit/web/bootstrap.py", line 347, in run
    if asyncio.get_running_loop().is_running():
       ^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: no running event loop

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-p